In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import torchvision
from torchsummary import summary
from tensorboardX import SummaryWriter

import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import IPython.display as ipd
import pickle
from scipy.io import loadmat
from tqdm import tqdm
import time
from glob import glob

import models
import utils
import params
from transition_S import Conv_S

In [2]:
"""Settings"""
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = "cuda" if torch.cuda.is_available() else "cpu"

"""Set Tensorboard Writer"""
writer_path = os.path.join(os.getcwd(), "wavenet_linear")
plot_save_path = os.path.join(os.getcwd(), "wavenet_linear_plots")

if not os.path.isdir(writer_path):
    os.mkdir(writer_path)

if not os.path.isdir(plot_save_path):
    os.mkdir(plot_save_path)

writer = SummaryWriter(writer_path)

"""HYPER PARAMETERS"""
input_channels = 12
output_channels = 8
win_size = 512 # Set it after calculate receptive field is calcuted
hop_len = 128

In [3]:
"""Load Model"""
model = models.WaveNet(layer_size=10, stack_size=1).to(device)
summary(model, (19999, 12))
receptive_field = model.calc_receptive_fields(layer_size=10, stack_size=1)
win_size += receptive_field

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1            [-1, 64, 20000]           1,536
      CausalConv1d-2            [-1, 64, 19999]               0
     ResidualStack-3        [-1, 2, 128, 18976]               0
              ReLU-4           [-1, 128, 18976]               0
            Conv1d-5           [-1, 256, 18976]          33,024
              ReLU-6           [-1, 256, 18976]               0
            Conv1d-7           [-1, 256, 18976]          65,792
           LastNet-8           [-1, 256, 18976]               0
              ReLU-9           [-1, 128, 18976]               0
           Conv1d-10           [-1, 256, 18976]          33,024
             ReLU-11           [-1, 256, 18976]               0
           Conv1d-12           [-1, 256, 18976]          65,792
          LastNet-13           [-1, 256, 18976]               0
             ReLU-14           [-1, 128

In [4]:
print("================> Loading DATA <===================")
#Load Data Pickle
#with open(params.ACC_DATA, 'rb') as f:
#    acc_pickle = pickle.load(f)

#with open(params.SOUND_DATA, 'rb') as f:
#    sound_pickle = pickle.load(f)

#Load Dataset
#acc_data = np.concatenate(list(map(lambda x : utils.slice_window(x, win_size, hop_len), acc_pickle)), axis=0)
#sound_data = np.concatenate(list(map(lambda x : utils.slice_window(x, win_size, hop_len), sound_pickle)), axis=0)
#np.save(os.path.join(params.PATH, "train_acc_win_512_hop_128.npy"), acc_data)
#np.save(os.path.join(params.PATH, "train_sound_win_512_hop_128.npy"), sound_data)
#acc_data = np.load(params.ACC_NPY)
#sound_data = np.load(params.SOUND_NPY)
acc_data = np.load(os.path.join(os.getcwd(), "sample_acc.npy"))
sound_data = np.load(os.path.join(os.getcwd(), "sample_sound.npy"))
print("================> DATA Loaded Completed <===================")

================> Loading DATA <===================
================> DATA Loaded Completed <===================


In [5]:
"""PreProcess Data"""
print("================> DATA Preprocessing <===================")
mu_encoder = utils.mu_law_encoder()
transform = torchvision.transforms.Compose([
                                    torchvision.transforms.ToTensor(),
                                    mu_encoder()])
dataset = utils.Wavenet_Dataset(acc_data, sound_data, receptive_field, transform=transform)
print("================> DATA Preprocessed <===================")

================> DATA Preprocessing <===================


TypeError: forward() missing 1 required positional argument: 'x'

In [ ]:
BATCH = 1
EPOCH = 100
dataloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=BATCH)

In [ ]:
model.train()
x, y = next(iter(dataloader))
x = x.squeeze(1).float().to(device)
y = y.squeeze(1).float().to(device)

optimizer.zero_grad()
pred = model(x)

s_filtered = Conv_S(pred, device=device)

loss = loss_fn(s_filtered, y)
loss.backward()
optimizer.step()

In [ ]:
s_filtered = Conv_S(pred, device=device)

loss = loss_fn(s_filtered, y)
loss.backward()
optimizer.step()